## Graphical User Interface (GUI) programming

It can be tremendous fun to build a graphical user interface for your programs to aid your exploration. [```pyqtgraph```](https://pyqtgraph.readthedocs.io/en/latest/) is a library that makes this large and intimidating subject more straight forward. 

Here we will walk through installation and exploring the examples of the libraries before we make a small example that shows off a few of the powerful concepts in ```pyqtgraph```.

![Screen shot of our example using pyqtgraph](PyQtGraphExampleAppScreenshot.png)

### Install ```pyqtgraph```

Here is the [website with the installation instructions](https://pyqtgraph.readthedocs.io/en/latest/getting_started/installation.html). 

In [2]:
from IPython.display import IFrame
IFrame(src='https://pyqtgraph.readthedocs.io/en/latest/getting_started/installation.html', width=700, height=600)

Once you have it installed we want to explore at least some of the many examples that come with the library. 

In [13]:
import pyqtgraph.examples
pyqtgraph.examples.run()

Generating scalar field..
Generating scalar field..
Generating isosurface..
Generating isosurface..


qt.pointer.dispatch: delivering touch release to same window QWindow(0x0) not QWidgetWindow(0x7fb7bc256e70, name="QSplitterClassWindow")
qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=1259.34,-755.004 gbl=1259.34,-755.004 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-1259.34,755.004 last=-1259.34,755.004 Δ 1259.34,-755.004) : no target window
qt.pointer.dispatch: delivering touch release to same window QWindow(0x0) not QWidgetWindow(0x7fb7bc256e70, name="QSplitterClassWindow")
qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=1239.26,-612.301 gbl=1239.26,-612.301 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-1239.26,612.301 last=-1239.26,612.301 Δ 1239.26,-612.301) : no target window
qt.pointer.dispatch: delivering touch release to same window QWindow(0x0) not QWidgetWindow(0x7fb7bc256e70, name="QSplitterClassWindow")
qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=1106.39,-901.504 gbl=1106.39,-901.504 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-110

: 

### One simple interactive plotting application

In this next example note that we have a single function ```sin_curve``` that has all named parameters. With one single command ```interact``` the library extracts all of those as parameters and makes them available via the ```ParameterTree``` widget in a list view such that they all can individually be modified. 

So for any function that you have you can wrap it analogously to generate a graphical user interface for it. 

In [4]:
# pyqtgraph example app
from pyqtgraph.Qt import QtWidgets
import pyqtgraph as pg
from pyqtgraph.parametertree import Parameter, ParameterTree, interact
from pyqtgraph.dockarea.Dock import Dock
from pyqtgraph.dockarea.DockArea import DockArea
import numpy as np

desc = """ Plotting a sine curve 
   y = A sin(k x + phi) 
 in the interval xmin...xmax
 using N points
 labeling the axes xlabel and ylabel.

 Explore the plot it has interactive features. 
 Double clicking on the titles of the docks splits
 them off as separate windows.  
"""
def sin_curve(A=5, k=6.28, phi=0, xmin=0, xmax=2*np.pi, 
              N=300,xlabel="space",ylabel="amplitude"):
    x = np.linspace(xmin,xmax, N)    
    pw.clear()
    pw.plot(x, A*np.sin(k*x + phi), pen=(150,150,255), name="Blue curve")
    pw.setLabel('bottom', xlabel, units='')
    pw.setLabel('left', ylabel, units='')
    return

## One line of code, no name/value duplication
params = interact(sin_curve)

## makes app 
app = pg.mkQApp()
## Define a top-level widget to hold everything
win = QtWidgets.QMainWindow()
win.setWindowTitle('PyQtGraph example for interactive plotting')
area = DockArea()
win.setCentralWidget(area)
win.resize(1000,500)

## Create docks, place them into the window one at a time.
## Note that size arguments are only a suggestion; docks will still have to
## fill the entire dock area and obey the limits of their internal widgets.
d0 = Dock("Description", closable=True)
d1 = Dock("Dock1 - parameters")     ## 
d2 = Dock("Dock2 - Plot", size=(30,40))

area.addDock(d2, 'right')     ## place plot at right edge of dock area
area.addDock(d0,'left',d2)    ## description on the left
area.addDock(d1, 'bottom', d0)  ## place parameters below description

tree = ParameterTree()
tree.setParameters(params)

text = QtWidgets.QLabel(desc)

## plot window
pw = pg.PlotWidget()

## Add widgets to the docks
d1.addWidget(tree)  
d2.addWidget(pw)  
d0.addWidget(text)

## Display the widget as a new window
win.show()

pg.exec()

0

Now let's create a visualization of one of our FFt based solvers

In [9]:
 """
Demonstrate use of GLLinePlotItem to draw cross-sections of a surface.
"""

import numpy as np

import pyqtgraph as pg
import pyqtgraph.opengl as gl

a = 0.5    # Thermal diffusivity constant
L = 2     # Length of domain
Tfinal = 2.

N = 1024 # Number of spatial discretization points
Nt = 200 # Number of time outputs

dx = L/N
x = np.arange(-L/2,L/2,dx) # Define x domain

# Define discrete wave-numbers
kappa = 2*np.pi*np.fft.fftfreq(N, d=dx)

# Initial condition
u0 = np.zeros_like(x)    # same shape as spatial grid
u0[np.abs(x)<=0.25] = 1  # square shape pulse 
u0hat = np.fft.fft(u0)   # ICs in Fourier space

# Simulate in Fourier frequency domain
t = np.linspace(0, Tfinal, Nt)

def heat_analytic_fourier(u0hat, kappa, a, t):
    return  u0hat * np.exp(-(kappa**2 * a**2 * t))

ua = np.zeros((N,Nt))
uahat = np.zeros(N)

# Fourier transform the solution back to configuration space 
for i, ct in enumerate(t):
    uahat = heat_analytic_fourier(u0hat, kappa, a, ct)
    ua[:,i] = np.fft.ifft(uahat).real


app = pg.mkQApp("GLLinePlotItem Example")
w = gl.GLViewWidget()
w.show()
w.setWindowTitle('pyqtgraph example: GLLinePlotItem')
w.setCameraPosition(distance=8)

gx = gl.GLGridItem()
gx.rotate(90, 0, 1, 0)
gx.translate(-10, 0, 0)
w.addItem(gx)
gy = gl.GLGridItem()
gy.rotate(90, 1, 0, 0)
gy.translate(0, -10, 0)
w.addItem(gy)
gz = gl.GLGridItem()
gz.translate(0, 0, -10)
w.addItem(gz)

for i in range(Nt):
    yi = t[i]
    d = x
    z = ua[:,i]
    pts = np.column_stack([x, np.full_like(x, yi), z])
#    plt = gl.GLLinePlotItem(pos=pts, color=pg.mkColor((Nt/2-i,Nt*1.3)), width=4, antialias=True)
    plt = gl.GLLinePlotItem(pos=pts, color=pg.mkColor(200,200,200,150), width=4, antialias=True)
    w.addItem(plt)

if __name__ == '__main__':
    pg.exec()


: 

### In class exercise:

Modify the 3D OpenGL example above so that it also uses a ```ParameterTree``` widget and allows to interactively change some parameters of the plot. You should be able to reuse most concepts from the first example above for the ```sine_curve``` application.

In [1]:
 """
Demonstrate use of GLLinePlotItem to draw cross-sections of a surface.
"""
import numpy as np
import pyqtgraph as pg
import pyqtgraph.opengl as gl
from pyqtgraph.Qt import QtWidgets
from pyqtgraph.parametertree import Parameter, ParameterTree, interact
from pyqtgraph.dockarea.Dock import Dock
from pyqtgraph.dockarea.DockArea import DockArea

def heat_equation_gui(a=0.5, L=2,Tfinal=2.,trans=0.5,N=1024,Nt=200):

    dx = L/N
    x = np.arange(-L/2,L/2,dx) # Define x domain

    # Define discrete wave-numbers
    kappa = 2*np.pi*np.fft.fftfreq(N, d=dx)

    # Initial condition
    u0 = np.zeros_like(x)    # same shape as spatial grid
    u0[np.abs(x)<=0.25] = 1  # square shape pulse 
    u0hat = np.fft.fft(u0)   # ICs in Fourier space

    # Simulate in Fourier frequency domain
    t = np.linspace(0, Tfinal, Nt)

    def heat_analytic_fourier(u0hat, kappa, a, t):
        return  u0hat * np.exp(-(kappa**2 * a**2 * t))

    ua = np.zeros((N,Nt))
    uahat = np.zeros(N)

    # Fourier transform the solution back to configuration space 
    for i, ct in enumerate(t):
        uahat = heat_analytic_fourier(u0hat, kappa, a, ct)
        ua[:,i] = np.fft.ifft(uahat).real

    w.clear()
    for i in range(Nt):
        yi = t[i]
        d = x
        z = ua[:,i]
        pts = np.column_stack([x, np.full_like(x, yi), z])
    #    plt = gl.GLLinePlotItem(pos=pts, color=pg.mkColor((Nt/2-i,Nt*1.3)), width=4, antialias=True)
        plt = gl.GLLinePlotItem(pos=pts, color=pg.mkColor(200,200,200,np.int64(255*trans)), width=4, antialias=True)
        w.addItem(plt)

    return 

## One line of code, no name/value duplication
params = interact(heat_equation_gui)


app = pg.mkQApp("GLLinePlotItem Example")
w = gl.GLViewWidget()

gx = gl.GLGridItem()
gx.rotate(90, 0, 1, 0)
gx.translate(-10, 0, 0)
w.addItem(gx)
gy = gl.GLGridItem()
gy.rotate(90, 1, 0, 0)
gy.translate(0, -10, 0)
w.addItem(gy)
gz = gl.GLGridItem()
gz.translate(0, 0, -10)
w.addItem(gz)
w.setCameraPosition(distance=8)

# Populate w with the standard solution

ua = heat_equation_gui()

## Define a top-level widget to hold everything
win = QtWidgets.QMainWindow()
win.setWindowTitle('PyQtGraph example for interactive plotting')
area = DockArea()
win.setCentralWidget(area)
win.resize(1000,500)

## Create docks, place them into the window one at a time.
## Note that size arguments are only a suggestion; docks will still have to
## fill the entire dock area and obey the limits of their internal widgets.
d0 = Dock("Description", closable=True)
d1 = Dock("Dock1 - parameters")     ## 
d2 = Dock("Dock2 - Plot", size=(30,40))

area.addDock(d2, 'right')     ## place plot at right edge of dock area
area.addDock(d0,'left',d2)    ## description on the left
area.addDock(d1, 'bottom', d0)  ## place parameters below description

tree = ParameterTree()
tree.setParameters(params)

desc = """ Plotting solution to the heat equation 
"""

text = QtWidgets.QLabel(desc)

## Add widgets to the docks
d1.addWidget(tree)  
d2.addWidget(w)  
d0.addWidget(text)

## Display the widget as a new window
win.show()

if __name__ == '__main__':
    pg.exec()
